In [9]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer as DV
from sklearn.feature_selection import *
from sklearn.preprocessing import MinMaxScaler as MMS,LabelEncoder as LE
from sklearn.decomposition import PCA
from sklearn.metrics import *
from sklearn.kernel_ridge import *
from sklearn import tree
from sklearn.ensemble import *
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.svm import LinearSVC, SVC, OneClassSVM
from sklearn.model_selection import *
from sklearn.linear_model import *
from collections import defaultdict
from sklearn.neural_network import MLPClassifier
from encoder import Encoder
from imblearn.over_sampling import SMOTE, ADASYN


In [8]:

df = pd.read_csv('/Users/bjayaram/Documents/Kaggle/criminal/criminal_train.csv')
label_cols = u'Criminal'
data_columns=list(df.columns)
data_columns.remove('PERID')
data_columns.remove('Criminal')
print data_columns


y_orig = df[label_cols].values
x_orig = df[data_columns].values

non_crim_indices = np.where(y_orig == 0)
crim_indices = np.where(y_orig == 1)
print len(non_crim_indices[0])

x_non_crim = x_orig[non_crim_indices]
y_non_crim = y_orig[non_crim_indices]

print x_non_crim, y_non_crim
print y_orig.sum(), len(y_orig)

['IFATHER', 'NRCH17_2', 'IRHHSIZ2', 'IIHHSIZ2', 'IRKI17_2', 'IIKI17_2', 'IRHH65_2', 'IIHH65_2', 'PRXRETRY', 'PRXYDATA', 'MEDICARE', 'CAIDCHIP', 'CHAMPUS', 'PRVHLTIN', 'GRPHLTIN', 'HLTINNOS', 'HLCNOTYR', 'HLCNOTMO', 'HLCLAST', 'HLLOSRSN', 'HLNVCOST', 'HLNVOFFR', 'HLNVREF', 'HLNVNEED', 'HLNVSOR', 'IRMCDCHP', 'IIMCDCHP', 'IRMEDICR', 'IIMEDICR', 'IRCHMPUS', 'IICHMPUS', 'IRPRVHLT', 'IIPRVHLT', 'IROTHHLT', 'IIOTHHLT', 'HLCALLFG', 'HLCALL99', 'ANYHLTI2', 'IRINSUR4', 'IIINSUR4', 'OTHINS', 'CELLNOTCL', 'CELLWRKNG', 'IRFAMSOC', 'IIFAMSOC', 'IRFAMSSI', 'IIFAMSSI', 'IRFSTAMP', 'IIFSTAMP', 'IRFAMPMT', 'IIFAMPMT', 'IRFAMSVC', 'IIFAMSVC', 'IRWELMOS', 'IIWELMOS', 'IRPINC3', 'IRFAMIN3', 'IIPINC3', 'IIFAMIN3', 'GOVTPROG', 'POVERTY3', 'TOOLONG', 'TROUBUND', 'PDEN10', 'COUTYP2', 'MAIIN102', 'AIIND102', 'ANALWT_C', 'VESTR', 'VEREP']
42543
[[  4.00000000e+00   2.00000000e+00   4.00000000e+00 ...,   3.88480600e+03
    4.00260000e+04   1.00000000e+00]
 [  4.00000000e+00   1.00000000e+00   2.00000000e+00 ..., 

In [3]:

smote = SMOTE(ratio='minority', k_neighbors= 5000, kind='svm', n_jobs=16)
# adasyn = ADASYN( n_jobs=24, n_neighbors=5)
pca=SelectKBest(chi2, k=5)
mms = MMS()
x_train, y_train = smote.fit_sample(x_train, y_train)
x_train = pca.fit_transform(mms.fit_transform(x_train, y_train), y_train)

print "SMOTE sample generated"


In [7]:
print x_train.shape, y_train.shape
model1=RandomForestClassifier(n_estimators=1000, n_jobs=-1)
# C=map(lambda x:10**x,range(-100,100,10))
# model = LogisticRegressionCV(Cs=C, penalty='l1', solver='liblinear', n_jobs=-1)
# model = MLPClassifier(hidden_layer_sizes=(100,50))
model = GridSearchCV(SVC(),n_jobs=16, scoring = 'f1',  cv = 3, param_grid={'kernel':['rbf', 'poly','sigmoid'],'C':range(1,100,10), 'class_weight':['balanced']})
# model = GridSearchCV(RandomForestClassifier(), cv = 5, n_jobs=24, param_grid={'n_estimators':range(3000,3001),'class_weight':['balanced']})

model.fit(x_train, y_train)
print "Grid best scores = ",model.best_score_
print "Grid all scores = ",model.cv_results_



df = pd.read_csv('/Users/bjayaram/Documents/Kaggle/criminal/criminal_test.csv')
label_cols = u'Criminal'
data_columns=list(df.columns)
data_columns.remove('PERID')
x_test = df[data_columns].values
x_test = pca.transform(mms.transform(x_test))

y_test = model.predict(x_test)
res= pd.DataFrame()
res['PERID']=df['PERID']
res['Criminal']=y_test
res.to_csv('/Users/bjayaram/Documents/Kaggle/criminal/results.csv', index=None)


(85085, 5) (85085,)


In [14]:
model = OneClassSVM()
# model = GridSearchCV(OneClassSVM,n_jobs=16, scoring = 'f1',  cv = 3, param_grid={'kernel':['rbf', 'poly','sigmoid'],'C':range(1,100,10)})

In [15]:
model.fit(x_non_crim)

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma='auto', kernel='rbf',
      max_iter=-1, nu=0.5, random_state=None, shrinking=True, tol=0.001,
      verbose=False)